## Assignment 1 Kai Foerster (ID: 214288)

In [117]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import zipfile
import os


In [118]:
# 1.1 Choose one of the sessions, and retrieve it using R or Python.

# Session 27 September 2023
url = "https://www.bundestag.de/resource/blob/968690/5d723616da1ea3ca054e8da604ff1004/20124-data.xml"

# Send a GET request to the server and store the response
r = requests.get(url)

soup = BeautifulSoup(r.content)

c:\Users\kaius\.virtualenvs\TaD-programming-excercises-FCqiZMWn\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [132]:
#1.2 Using a scraper, get a list of all the elements.
rede_elements = soup.find_all('rede')
rede_elements

[<rede id="ID2012400100">
 <p klasse="redner"><a id="r1"></a><redner id="11004097"><name><vorname>Christian</vorname><nachname>Lindner</nachname><rolle><rolle_lang>Bundesminister der Finanzen</rolle_lang><rolle_kurz>Bundesminister BMF</rolle_kurz></rolle></name></redner>Christian Lindner, Bundesminister der Finanzen:</p>
 <p klasse="J_1">Frau Präsidentin! Liebe Kolleginnen und Kollegen! Ich will mich zunächst bei der Innenministerin, Nancy Faeser, bedanken, dass sie in der vergangenen Woche kurzfristig eingesprungen ist, als ich positiv war, eine Covid-Infektion hatte.</p>
 <p klasse="J">Ich will drei Punkte nennen.</p>
 <p klasse="J">Erster Punkt. Die wirtschaftliche Entwicklung in unserem Land ist unbefriedigend. Hierbei sind zum einen konjunkturelle Belastungsfaktoren, zum anderen aber auch strukturelle Defizite unserer Wettbewerbsfähigkeit, die wir seit vielen Jahren kennen, zu nennen. Die Bundesregierung geht diese entschlossen an, von A wie „Arbeitskräfte“ bis P wie „Planungs- un

In [120]:
# 1.3 For each element, get the name of the speaker, and a single string containing everything that they said. 
# Put this into a dataframe.

# Initialize a list to store the extracted info
data = []

for rede in rede_elements:
    # Extract the paragraph describing the speaker
    vorname_element = rede.select('vorname')
    nachname_element = rede.select('nachname')
    
    # Check if vorname and nachname elements are not empty, and extract text from them
    vorname = vorname_element[0].text if vorname_element else ""
    nachname = nachname_element[0].text if nachname_element else ""
    
    # Extract the speech paragraphs with klasse="J", "J_1", or "O"
    speech_paragraphs = rede.select('p', class_=['J', 'J_1', 'O'])
    
    # Combine all speech paragraphs into one string, skipping the first paragraph if there are more than one
    speech_text = ' '.join([p.get_text(strip=True) for p in speech_paragraphs[1:]]) 
    
    # Append the extracted information to the data list
    data.append({
        "Name": vorname + " " + nachname,
        "Speech": speech_text
    })

df = pd.DataFrame.from_dict(data)
df    

,Name,Speech
0,Christian Lindner,Frau Präsidentin! Liebe Kolleginnen und Kolleg...
1,Svenja Schulze,"Sehr geehrte Frau Präsidentin, das freut uns a..."
2,Mathias Middelberg,"Frau Präsidentin, herzlichen Dank für das Wort..."
3,Christian Lindner,"Vielen Dank, Frau Präsidentin. – Lieber Herr K..."
4,Mathias Middelberg,Auch wir finden den angebotsorientierten Ansat...
...,...,...
159,Manfred Todtenhausen,Frau Präsidentin! Liebe Kolleginnen! Liebe Kol...
160,Martina Stamm-Fibich,Sehr geehrte Frau Präsidentin! Liebe Kolleginn...
161,Dirk Brandes,"Vielen Dank, Frau Präsidentin, dass Sie die Ku..."
162,Daniela Ludwig,Frau Präsidentin! Liebe Kolleginnen und Kolleg...


In [121]:
#2.1 Choose a politician, and print the number of speeches they made in this session

# Group by 'Name', count the occurrences, and reset the index
grouped_df = df.groupby('Name').size().reset_index(name='count')

# Sort the DataFrame based on the 'count' column in descending order
sorted_df = grouped_df.sort_values(by='count', ascending=False)

# Filter the rows where 'Name' is 'Christian Lindner'
filtered_df = sorted_df[sorted_df['Name'] == 'Christian Lindner']

print(filtered_df)

                 Name  count
14  Christian Lindner     27


In [122]:
# 2.2 Print the content of the first speech by the politician you choose.

# Filter rows where 'Name' is 'Svenja Schulze'
lindner_speech = df[df['Name'] == 'Christian Lindner']

# Access the first row and the second column
element = lindner_speech.iloc[0, 1]  # Remember, Python uses zero-based indexing. lindner_speech['Speech'].iloc[0] is an alternative

print(element)

Frau Präsidentin! Liebe Kolleginnen und Kollegen! Ich will mich zunächst bei der Innenministerin, Nancy Faeser, bedanken, dass sie in der vergangenen Woche kurzfristig eingesprungen ist, als ich positiv war, eine Covid-Infektion hatte. Ich will drei Punkte nennen. Erster Punkt. Die wirtschaftliche Entwicklung in unserem Land ist unbefriedigend. Hierbei sind zum einen konjunkturelle Belastungsfaktoren, zum anderen aber auch strukturelle Defizite unserer Wettbewerbsfähigkeit, die wir seit vielen Jahren kennen, zu nennen. Die Bundesregierung geht diese entschlossen an, von A wie „Arbeitskräfte“ bis P wie „Planungs- und Genehmigungsverfahren“, die wir beschleunigen wollen. In meinem Geschäftsbereich kommen zwei wichtige Gesetzgebungsvorhaben hinzu: zum einen das Wachstumschancengesetz, mit dem wir Forschungsförderung, Investitionen und Eigenkapitalbasis stärken, sowie das Zukunftsfinanzierungsgesetz, mit dem wir den Kapitalmarktzugang insbesondere für junge und innovative Unternehmen verbe

In [143]:
# 2.3 Process the list of speeches into a TFIDF matrix. What are the highest scoring terms in this matrix for the
# first speech by the politician you have chosen?

import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer
from nltk.tokenize import word_tokenize

# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Load German stop words
german_stop_words = set(stopwords.words('german'))

# Initialize German stemmer
stemmer = GermanStemmer()

# Define a function for preprocessing
def preprocess(text):
    # Tokenize and remove punctuation
    words = word_tokenize(text)
    words = [word for word in words if word.isalpha()]
    
    # Remove stop words and perform stemming
    return ' '.join(stemmer.stem(word) for word in words if word.lower() not in german_stop_words)

# Preprocess all speeches and store them in a list
processed_speeches = [preprocess(speech) for speech in df['Speech'] if isinstance(speech, str)]

# Compute TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_speeches)  # Input is the entire list of processed_speeches

# Extract top features based on tf-idf scores for the first speech
features = vectorizer.get_feature_names_out()
first_speech_tfidf = tfidf_matrix[0]  # Get the tfidf vector for the first speech, which happens to be Linder's first speech

# Create a dictionary with features and their corresponding tf-idf scores
data = []
for col, term in enumerate(features):
    data.append((term, first_speech_tfidf[0, col]))

ranking = pd.DataFrame(data, columns=['term', 'rank'])
sorted_ranking = ranking.sort_values('rank', ascending=False)

# Display top 11 features
print(sorted_ranking.head(11))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kaius\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                       term      rank
2670               kontroll  0.208464
2372              inflation  0.181213
3705  schlepperkriminalitat  0.167296
4974                   zoll  0.154764
2401              insbesond  0.130280
1047                deshalb  0.115673
3894                   sowi  0.114607
4412                verbess  0.114607
4869             wirtschaft  0.113232
662                  bereit  0.107639
3130                   nenn  0.105020


In [124]:
# 2.4 Using the resource “Stammdaten aller Abgeordneten seit 1949 im XML-Format”, retrieve the records
# pertaining to your chosen politician and print the information they contain.

# The url we use to download the zip file

url = 'https://www.bundestag.de/resource/blob/472878/a4859899e44a7cab1a8233e5dd69f2f3/MdB-Stammdaten-data.zip'  

# download the zip file
response = requests.get(url)

# ensure the response status is OK
if response.status_code == 200:
    with open("temp.zip", "wb") as f:
        f.write(response.content)
    
    # extract the content of the zip file
    with zipfile.ZipFile("temp.zip", 'r') as zip_ref:
        # if you know the name of the file in the zip file, you can extract it directly, but we don't
        # zip_ref.extract('filename', 'temp')
        
        # to extract all files in the zip file
        zip_ref.extractall("temp")
        
        xml_file_names = zip_ref.namelist()  # Get the list of names of files in the zip file

# log the names of the files in the zip file
print(f"Extracted the following files: {xml_file_names}")

# construct the path to the desired XML file
xml_file_path = os.path.join("temp", xml_file_names[1]) 


Extracted the following files: ['MDB_STAMMDATEN.DTD', 'MDB_STAMMDATEN.XML']


In [125]:
#First we parse the file
tree = etree.parse(xml_file_path)

In [126]:
# Define a function to convert XML tree into a dictionary
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = {}
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                if k in dd:
                    if type(dd[k]) is list:
                        dd[k].append(v)
                    else:
                        dd[k] = [dd[k], v]
                else:
                    dd[k] = v
        d = {t.tag: dd}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
                d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d


In [127]:
# Convert the XML tree to a dictionary
xml_dict = etree_to_dict(tree.getroot())

In [128]:
# Context: This is how I was able to access the first and last name, so now the question is how to loop over
# first_name = xml_dict['DOCUMENT']['MDB'][1].get('NAMEN', {}).get('NAME', {}).get('VORNAME', "")
# last_name = xml_dict['DOCUMENT']['MDB'][1].get('NAMEN', {}).get('NAME', {}).get('NACHNAME', "")
# additionally it seems like not all entries in list of MPs have the same dictionary format, meaning that
# we need to try-catch the whole thing so the loop can continue running.

# Initialize an empty list to store the entries for Christian Lindner
lindner_entries = []

# Getting the list of entries
entries = xml_dict.get('DOCUMENT', {}).get('MDB', [])
# Note the first MP in the list would be accessed through entries[0]

# Looping over each entry in the list
for entry in entries:
    try:
        # Access the nested dictionaries to get the name information
        name_info = entry.get('NAMEN', {}).get('NAME', {})
        first_name = name_info.get('VORNAME', "")
        last_name = name_info.get('NACHNAME', "")
        
        # Check if the name matches Christian Lindner
        if first_name == "Christian" and last_name == "Lindner":
            # If yes, then append the entry to lindner_entries
            lindner_entries.append(entry)
    except AttributeError:
        # Handle the case where entry is not a dictionary and does not have .get() method
        continue

# lindner_entries now contain all the entries where the name is Christian Lindner
print(lindner_entries)



[{'ID': '11004097', 'NAMEN': {'NAME': {'NACHNAME': 'Lindner', 'VORNAME': 'Christian', 'ORTSZUSATZ': None, 'ADEL': None, 'PRAEFIX': None, 'ANREDE_TITEL': None, 'AKAD_TITEL': None, 'HISTORIE_VON': '27.10.2009', 'HISTORIE_BIS': None}}, 'BIOGRAFISCHE_ANGABEN': {'GEBURTSDATUM': '07.01.1979', 'GEBURTSORT': 'Wuppertal', 'GEBURTSLAND': None, 'STERBEDATUM': None, 'GESCHLECHT': 'männlich', 'FAMILIENSTAND': 'geschieden', 'RELIGION': 'konfessionslos', 'BERUF': 'Mitglied des Bundestages', 'PARTEI_KURZ': 'FDP', 'VITA_KURZ': 'Abitur 1998. Major d. R. Studium der Politikwissenschaft, des Öffentlichen Rechts und der Philosophie an der Univ. Bonn, Magister Artium. 1997/2004 in der Werbebranche selbstständig. 2000/11 Teilhaber eines Start-up-Unternehmens. Mitgl. der FDP seit 1995, seit 1998 Mitgl. NRW Landesvorstand der FDP, 2004/10 Generalsekretär der FDP NRW, 2009/11 Generalsekretär der FDP, 2012/17 Landesvors. FDP NRW, seit Dez. 2013 FDP-Bundesvors. MdL Nordrhein-Westfalen 2000/09 und 2012/17, 2012/17